In [27]:
import pandas as pd
import mysql.connector
import numpy as np
from sklearn import tree

def import_data():
    con  = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="000012300"
        )
    cur = con.cursor()
    cur.execute("USE mini_data")
    cur.execute("SELECT code from symptoms ORDER BY code")
    ##variables
    diseaseCodes = []
    symptomCode = []
    relations=[]

    for i in cur:
        symptomCode.append(i[0])

    cur.execute("SELECT code from diseases ORDER BY code")
    for i in cur:
        diseaseCodes.append(i[0])

    relations =  np.zeros((len(diseaseCodes), len(symptomCode)))

    for i in diseaseCodes:
        cur.execute("SELECT symptom_code from disease_symptom WHERE disease_code= '"+i+" '")
        for k in cur:
            relations[diseaseCodes.index(i)][symptomCode.index(k[0])]=1
        
    # for i in range(len(diseaseCodes)):
    #     print(diseaseCodes[i],end="\t")
    #     for j in range(len(symptomCode)):
    #         print(relations[i][j],end =" ")
    return diseaseCodes,symptomCode,relations

In [28]:
from sklearn.tree.export import export_text
diseaseCodes,symptomCode,relations = import_data()
clf = tree.DecisionTreeClassifier(criterion='entropy', splitter='best')
clf = clf.fit(relations,diseaseCodes)

print(clf.predict([relations[10]]))
print(diseaseCodes[10])

etx = export_text(clf,feature_names=symptomCode)
print(etx)

['C0004096']
C0004096
|--- C0392680 <= 0.50
|   |--- C0030193 <= 0.50
|   |   |--- C0015967 <= 0.50
|   |   |   |--- C0085631 <= 0.50
|   |   |   |   |--- C0241526 <= 0.50
|   |   |   |   |   |--- C0000737 <= 0.50
|   |   |   |   |   |   |--- C0043096 <= 0.50
|   |   |   |   |   |   |   |--- C0010200 <= 0.50
|   |   |   |   |   |   |   |   |--- C1299586 <= 0.50
|   |   |   |   |   |   |   |   |   |--- C0028081 <= 0.50
|   |   |   |   |   |   |   |   |   |   |--- C0009024 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- truncated branch of depth 2
|   |   |   |   |   |   |   |   |   |   |--- C0009024 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0003537
|   |   |   |   |   |   |   |   |   |--- C0028081 >  0.50
|   |   |   |   |   |   |   |   |   |   |--- C0520966 <= 0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0005001
|   |   |   |   |   |   |   |   |   |   |--- C0520966 >  0.50
|   |   |   |   |   |   |   |   |   |   |   |--- class: C0034067


In [33]:
#utility function to diaplay decision tree
def plot_tree(clf,disease,symptoms):
    from sklearn.externals.six import StringIO  
    from IPython.display import Image  
    from sklearn.tree import export_graphviz
    import pydotplus
    dot_data = StringIO()
    export_graphviz(clf, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True, feature_names = symptoms)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    graph.write_png('diabetes.png')
    Image(graph.create_png())

In [6]:
print(clf.get_depth())

12


In [37]:
plot_tree(clf,diseaseCodes,symptomCode)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.834084 to fit

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.834084 to fit



In [ ]:
#to generate decision rule

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    print "def tree({}):".format(", ".join(feature_names))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print "{}if {} <= {}:".format(indent, name, threshold)
            recurse(tree_.children_left[node], depth + 1)
            print "{}else:  # if {} > {}".format(indent, name, threshold)
            recurse(tree_.children_right[node], depth + 1)
        else:
            print "{}return {}".format(indent, tree_.value[node])

    recurse(0, 1)

In [25]:
#to show the symptom codes for any disease 
con  = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="000012300"
    )
cur = con.cursor()
cur.execute("USE mini_data")
querry = "SELECT symptom_code from disease_symptom WHERE disease_code = '" +diseaseCodes[0]+"'"
print(querry)
cur.execute(querry)
for i in cur:
    print (i)


SELECT symptom_code from disease_symptom WHERE disease_code = 'C0001175'
('C0015967',)
('C0028081',)
('C0233308',)
('C0010200',)
('C0043096',)
('C0085593',)
('C0011991',)
('C0008033',)
('C0376405',)
('C0231835',)
('C0239134',)
('C0026827',)
('C0424000',)
